In [1]:
from deep_sort_app import run
import time

d:\deepsort\Modular-DeepSort\packages/deep-person-reid\torchreid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
mot_files = [
    "testing_data/KITTI-17/",
    "testing_data/MOT16-09/",
    "testing_data/MOT16-11/",
    "testing_data/PETS09-S2L1/",
    "testing_data/TUD-Campus/",
    "testing_data/TUD-Stadtmitte/",
]
detection_models = [
    "original",
    "yolov5nano",
    # "yolov5small",
    # "yolov5medium",
    # "yolov5large",
    "yolov5extralarge",
    "yolov10nano",
    # "yolov10small",
    # "yolov10medium",
    "yolov10balanced",
    # "yolov10large",
    "yolov10extralarge",
    "nanodet",
]
feature_generator_models = [
    "original",
    "dpreid_shufflenet",
    "dpreid_mlfn",
    "dpreid_mobilenetv2_1_0",
    # "dpreid_mobilenetv2_1_4",
    "dpreid_osnet_ibn_x1_0",
    "dpreid_osnet_ain_x1_0",
    # "dpreid_osnet_ain_x0_75",
    # "dpreid_osnet_ain_x0_5",
    # "dpreid_osnet_ain_x0_25",
    "dpreid_osnet_x1_0",
    # "dpreid_osnet_x0_75",
    # "dpreid_osnet_x0_5",
    # "dpreid_osnet_x0_25",
]

In [3]:
hyperparameters = {
    "min_confidence": 0.8,
    "nms_max_overlap": 1.0,
    "min_detection_height": 0,
    "max_cosine_distance": 0.2,
    "nn_budget": None
}
display = False

In [5]:
with open("results_original_hyperparameters.md", "a") as file:
    for d_model, fg_model in ((x, y) for x in detection_models for y in feature_generator_models):
        mean_metrics = {
            "precision": 0,
            "recall": 0,
            "f1-score": 0,
            "hota": 0,
            "fps": 0
        }
        print(d_model, fg_model)
        file.write("Detection Model: " + d_model + "\n")
        file.write("Feature Generation Model: " + fg_model + "\n")
        video_ = []
        precision_ = []
        recall_ = []
        f1_score_ = []
        hota_ = []
        fps_ = []
        for seq in mot_files:
            t0 = time.time()
            metrics = run(
                seq, 
                None, 
                None, 
                hyperparameters["min_confidence"],
                hyperparameters["nms_max_overlap"],
                hyperparameters["min_detection_height"],
                hyperparameters["max_cosine_distance"],
                hyperparameters["nn_budget"],
                d_model,
                fg_model,
                display
            )
            t1 = time.time()
            metrics["fps"] = metrics["frames"] / (t1 - t0)
            print(seq, metrics)
            # file.write("Video: " + seq + "\n")
            # file.write("Precision: " + str(round(metrics["precision"], 2)) + "\n")
            # file.write("Recall: " + str(round(metrics["recall"], 2)) + "\n")
            # file.write("F1-score: " + str(round(metrics["f1-score"], 2)) + "\n")
            # file.write("Hota: " + str(round(metrics["hota"], 2)) + "\n") 
            # file.write("FPS: " + str(round(metrics["fps"], 2)) + "\n") 
            video_.append(seq)
            precision_.append(str(round(metrics["precision"], 2)))
            recall_.append(str(round(metrics["recall"], 2)))
            f1_score_.append(str(round(metrics["f1-score"], 2)))
            hota_.append(str(round(metrics["hota"], 2)))
            fps_.append(str(round(metrics["fps"], 2)))
            mean_metrics["precision"] += metrics["precision"]
            mean_metrics["recall"] += metrics["recall"]
            mean_metrics["f1-score"] += metrics["f1-score"]
            mean_metrics["hota"] += metrics["hota"]
            mean_metrics["fps"] += metrics["fps"]
        mean_metrics["precision"] = round(mean_metrics["precision"] / len(mot_files), 2)
        mean_metrics["recall"] = round(mean_metrics["recall"] / len(mot_files), 2)
        mean_metrics["f1-score"] = round(mean_metrics["f1-score"] / len(mot_files), 2)
        mean_metrics["hota"] = round(mean_metrics["hota"] / len(mot_files), 2)
        mean_metrics["fps"] = round(mean_metrics["fps"] / len(mot_files), 2)
        file.write("Metrics Table:")
        file.write("| Video | Precision | Recall | F1-score | HOTA | FPS |\n")
        file.write("| --- | --- | --- | --- | --- | --- |\n")
        for i in range(len(video_)):
            file.write("| " + video_[i] + " | " + precision_[i] + " | " + recall_[i] + " | " + f1_score_[i] + " | " + hota_[i] + " | " + fps_[i] + " |\n")
        file.write("| ALL | " + str(mean_metrics["precision"]) + " | " + str(mean_metrics["recall"]) + " | " + str(mean_metrics["f1-score"]) + " | " + str(mean_metrics["hota"]) + " | " + str(mean_metrics["fps"]) + " |\n")
        file.write("---\n")

        # file.write("Mean metrics for all videos\n")
        # file.write("Precision: " + str(round(mean_metrics["precision"], 2)) + "\n")
        # file.write("Recall: " + str(round(mean_metrics["recall"], 2)) + "\n")
        # file.write("F1-score: " + str(round(mean_metrics["f1-score"], 2)) + "\n")
        # file.write("Hota: " + str(round(mean_metrics["hota"], 2)) + "\n") 
        # file.write("FPS: " + str(round(mean_metrics["fps"], 2)) + "\n") 
        # file.write("---------\n")

        print(mean_metrics)

original original
testing_data/KITTI-17/ {'precision': 0.9599271402550091, 'recall': 0.7715959004392386, 'f1-score': 0.8555194805194806, 'hota': 0.8084988883560865, 'frames': 145, 'fps': 51.324526270129404}


d:\deepsort\Modular-DeepSort\deep_sort\preprocessing.py:47: RuntimeWarning: overflow encountered in multiply
  area = (x2 - x1 + 1) * (y2 - y1 + 1)
d:\deepsort\Modular-DeepSort\deep_sort\preprocessing.py:66: RuntimeWarning: overflow encountered in multiply
  overlap = (w * h) / area[idxs[:last]]
d:\deepsort\Modular-DeepSort\deep_sort\preprocessing.py:66: RuntimeWarning: invalid value encountered in divide
  overlap = (w * h) / area[idxs[:last]]


testing_data/MOT16-09/ {'precision': 1.0, 'recall': 0.387102910405174, 'f1-score': 0.5581459133296763, 'hota': 0.6196377194614634, 'frames': 525, 'fps': 19.69502203511854}
testing_data/MOT16-11/ {'precision': 0.9997514292816306, 'recall': 0.43841290603880534, 'f1-score': 0.6095324695006442, 'hota': 0.6574320796793435, 'frames': 900, 'fps': 15.67242692273488}
testing_data/PETS09-S2L1/ {'precision': 0.7955116696588869, 'recall': 0.989946380697051, 'f1-score': 0.882142146127812, 'hota': 0.8945779441853692, 'frames': 795, 'fps': 18.165404810863272}
testing_data/TUD-Campus/ {'precision': 0.9314641744548287, 'recall': 0.8328690807799443, 'f1-score': 0.8794117647058822, 'hota': 0.824727597721162, 'frames': 71, 'fps': 42.386624985392906}
testing_data/TUD-Stadtmitte/ {'precision': 0.9201419698314108, 'recall': 0.8970588235294118, 'f1-score': 0.9084537888742883, 'hota': 0.8912429021703114, 'frames': 179, 'fps': 35.00878208040778}


TypeError: can only concatenate str (not "float") to str